# Import Required Libraries and Install Dependencies

In [ ]:
!pip install datasets
!pip install torch torchvision --upgrade
!pip install datasets transformers torch bert_score
!pip install tf-keras
!pip install torch torchvision accelerate

  Using cached torch-2.4.1-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl

  Using cached torch-2.5.1-cp310-cp310-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.21.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.4.127-py3-none-

# Load Hugging Face Model and Tokenizer

In [ ]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM

# Save Hugging Face token as an environment variable

os.environ["HF_TOKEN"] = "your_huggingface_token_here"

# Load tokenizer and model using the token argument
model_name = "microsoft/deberta-v3-large"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    token=os.environ["HF_TOKEN"],  # Use token instead of use_auth_token
    trust_remote_code=True
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=os.environ["HF_TOKEN"],  # Use token instead of use_auth_token
    trust_remote_code=True
)

print("Model and tokenizer loaded successfully!")


Model and tokenizer loaded successfully!


# Import Additional Libraries for Training

In [ ]:
import logging
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import torch
from torch.utils.data import Dataset

# Suppress unnecessary logs

In [ ]:
logging.getLogger("transformers.tokenization_utils_base").setLevel(logging.ERROR)
logging.getLogger("datasets.arrow_dataset").setLevel(logging.ERROR)

# Initialize Model

In [ ]:
if hasattr(model, "roberta"):
    torch.nn.init.xavier_uniform_(model.roberta.pooler.dense.weight)
    torch.nn.init.zeros_(model.roberta.pooler.dense.bias)

# Load and Prepare Dataset

In [ ]:
os.environ["HF_TOKEN"] = "your_huggingface_token_here"
data = load_dataset("anab/ACORN", split="train[:3000]")

def prepare_data(data):
    examples = []
    for i in range(len(data)):
        text = f"Question: {data['question'][i]} Answer: {data['choices'][i][data['label'][i]]} Explanation: {data['explanation'][i]}"
        ratings = data["voted_ratings"][i]
        label = [
            ratings.get("supports", 0),
            ratings.get("overall", 0),
            ratings.get("well_written", 0),
            ratings.get("related", 0),
            ratings.get("factual", 0),
            ratings.get("new_info", 0),
            ratings.get("unnecessary_info", 0),
            ratings.get("contrastive", 0),
        ]
        examples.append({"text": text, "label": label})
    return examples

formatted_data = prepare_data(data)
train_data, val_data = train_test_split(formatted_data, test_size=0.3, random_state=42)


# Define Custom Dataset Class

In [ ]:
class RatingsDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        encoding = self.tokenizer(
            item["text"],
            truncation=True,
            padding=True,
            return_tensors="pt"
        )
        labels = torch.tensor(item["label"], dtype=torch.float)
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": labels
        }

# Load Pretrained Model and Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")
model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/deberta-v3-large",
    num_labels=8,
    problem_type="regression"
)

# Create Data Loaders

In [ ]:
train_dataset = RatingsDataset(train_data, tokenizer)
val_dataset = RatingsDataset(val_data, tokenizer)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Define Metrics


In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.squeeze()
    mse = mean_squared_error(labels, predictions)
    r2 = r2_score(labels, predictions)
    return {"mean_squared_error": mse, "r2_score": r2}

# Define Training Arguments and Trainer

In [ ]:
training_args = TrainingArguments(
    output_dir="./results_deberta_regression",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=15,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs_deberta_regression",
    logging_strategy="steps",
    logging_steps=100,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="mean_squared_error",
    greater_is_better=False,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train and Save Model

In [ ]:
trainer.train()

output_dir = "./fine_tuned_deberta_regression"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"\nFine-tuning complete. Model and tokenizer saved to {output_dir}.")



# Export Fine-Tuned Model

In [ ]:
import shutil
from google.colab import files

model_directory = "./fine_tuned_deberta_regression"
shutil.make_archive("fine_tuned_deberta_regression", 'zip', model_directory)
files.download("fine_tuned_deberta_regression.zip")